In [1]:
pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 torch==1.11.0 GPUtil

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.3 requires absl-py~=0.10, but you have absl-py 1.0.0 which is incompatible.
tensorflow 2.6.3 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.3 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.6.3 requires wrapt~=1.12.1, but you have wrapt 1.14.0 which is incompatible.
tensorflow-transform 1.7.0 requires pyarrow<6,>=1, but you have pyarrow 7.0.0 which is incompatible.
tensorflow-transform 1.7.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,<2.9,>=1.15.5, but you have tensorflow 2.6.3 which is incompatible.
tensorflow-serving-api 2.8.0 requires tensorflow<3,>=2.8.0, but you have tensorflow 2.6.3 which i

In [2]:
# 1 导入必备的库
import copy
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# 3 Model Architecture
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        """
        初始化函数
        :param encoder: 编码器对象
        :param decoder: 解码器对象
        :param src_embed: 源数据嵌入函数
        :param tgt_embed: 目标数据嵌入函数
        :param generator: 类别生成器对象
        """
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        """
        将src, src_mask传入编码函数，得到结果后与src_mask, tgt和tgt_mask一同传给解码函数
        :param src: 源数据
        :param tgt: 目标数据
        :param src_mask: 源数据掩码张量
        :param tgt_mask: 目标数据掩码张量
        """
        memory = self.encode(src, src_mask)
        res = self.decode(memory, src_mask, tgt, tgt_mask)
        return res

    def encode(self, src, src_mask):
        """
        编码函数，使用src_embed对source做处理，然后和src_mask一起传给self.encoder
        """
        source_embeddings = self.src_embed(src)
        return self.encoder(source_embeddings, src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        """
        解码函数，使用tgt_embed对target做处理，然后和src_mask,tgt_mask,memory一起传给self.decoder
        """
        target_embeddings = self.tgt_embed(tgt)
        return self.decoder(target_embeddings, memory, src_mask, tgt_mask)


class Generator(nn.Module):
    """
    将线性层和softmax计算层一起实现， 把类的名字叫做Generator，生成器类
    """

    def __init__(self, d_model, vocab):
        """
        初始化函数
        :param d_model: 嵌入的维度
        :param vocab: vocab.size -> 词表的大小
        """
        super(Generator, self).__init__()
        self.proj = nn.Linear(in_features=d_model, out_features=vocab)

    def forward(self, x):
        """
        输入是上一层的输出张量x
        使用上一步得到的self.proj对x进行线性变化, 然后使用F中已经实现的log_softmax进行softmax处理。
        """
        softmax = F.log_softmax(self.proj(x), dim=-1)
        return softmax


# 3.1 Encoder and Decoder Stacks

# 3.1.1 Encoder
def clone(module, N):
    """
    用于克隆多份结构
    """
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class Encoder(nn.Module):
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clone(layer, N)  # 实现简单的克隆，在叠加在一起
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """
        将输入（和掩码）依次通过每一层。
        """
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


# 3.1.2 Layer Normalization and Residual Connections
class LayerNorm(nn.Module):

    def __init__(self, feature_size, eps=1e-6):
        """
        初始化函数
        :param feature_size: 词嵌入的维度
        :param eps: 防止分母为0，默认是1e-6
        """
        super(LayerNorm, self).__init__()
        # 使用nn.parameter封装，代表他们是模型的参数
        self.gamma = nn.Parameter(torch.ones(feature_size))  # 缩放参数向量 初始化为1张量
        self.beta = nn.Parameter(torch.zeros(feature_size))  # 平移参数向量 初始化为0张量
        self.eps = eps

    def forward(self, x):
        """
        1. 对输入变量x求其最后一个维度,即词嵌入维度的均值，并保持输出维度与输入维度一致
        2. 求最后一个维度的标准差，进行规范化：用x减去均值除以标准差
        3. 对结果乘以我们的缩放参数gamma, *表示点乘，加上位移参beta
        :param x: 来自上一层的输出
        """
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return (x - mean) / (std + self.eps) * self.gamma + self.beta


class SublayerConnection(nn.Module):
    """
    SublayerConnection类:实现子层连接结构.𝑥表示上一层添加了残差连接的输出，这一层添加了残差连接的输出需要将  𝑥  执行层级归一化，
    然后馈送到 Multi-Head Attention 层或全连接层，添加 Dropout 操作后可作为这一子层级的输出。最后将该子层的输出向量与输入向量相加得到下一层的输入。
    """

    def __init__(self, size, dropout):
        """
        :param size: 𝑑𝑚𝑜𝑑𝑒𝑙=512
        :param dropout: 丢弃参数
        """
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x, sublayer):
        # 原方案：先将x执行层级归一化
        # sublayer_out = sublayer(self.norm(x))
        # return x + self.dropout(sublayer_out)
        # 改进版本：取出norm 加快收敛速度
        sublayer_out = sublayer(x)
        sublayer_out = self.dropout(sublayer_out)
        return x + self.norm(sublayer_out)


# 3.1.3 Encoder Layer
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attention, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attention
        self.feed_forward = feed_forward
        self.sublayer = clone(module=SublayerConnection(size, dropout), N=2)  # 两次的跳过连接
        self.size = size

    def forward(self, x, mask):
        """
        第一个子层包括一个多头自注意力层和规范化层以及一个残差连接
        第二个子层包括一个前馈全连接层和规范化层以及一个残差连接
        """
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))  # 输入 Query、Key 和 Value 都为 x 就表示自注意力。
        z = self.sublayer[1](x, self.feed_forward)
        return z


# 3.1.4 Decoder

class Decoder(nn.Module):
    def __init__(self, layer, N):
        """
        :param layer: 解码器层layer
        :param N: 解码器层的个数N
        """
        super(Decoder, self).__init__()
        self.layers = clone(layer, N)  # 实现简单的克隆，在叠加在一起
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


# 3.1.5 Decoder Layer
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attention, src_attn, feed_forward, dropout):
        """
        :param self_attention: 多头自注意力对象，该注意力机制需要Q=K=V
        :param src_attn: 多头注意力对象，这里Q!=K=V
        :param dropout: dropout置0比率
        """
        super(DecoderLayer, self).__init__()
        self.self_attn = self_attention
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clone(module=SublayerConnection(size, dropout), N=3)  # 三次的跳过连接
        self.size = size

    def forward(self, x, memory, src_mask, tgt_mask):
        """
        :param x: 上一层的输入
        :param memory: 来自编码器层的语义存储变量
        :param src_mask: 源数据掩码张量
        :param tgt_mask: 目标数据掩码张量
        """

        """
        将x传入第一个子层结构，第一个子层结构的输入分别是x和self-attn函数，因为是自注意力机制，所以Q,K,V都是x，
        最后一个参数时目标数据掩码张量，这时要对目标数据进行遮掩，因为此时模型可能还没有生成任何目标数据。
        比如在解码器准备生成第一个字符或词汇时，我们其实已经传入了第一个字符以便计算损失，但是我们不希望在生成第一个字符时模型能利用这个信息，
        因此我们会将其遮掩，同样生成第二个字符或词汇时，模型只能使用第一个字符或词汇信息，第二个字符以及之后的信息都不允许被模型使用。
        """
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))  # 输入 Query、Key 和 Value 都为 x 就表示自注意力。
        """
        接着进入第二个子层，这个子层中常规的注意力机制，q是输入x;
        k,v是编码层输出memory，同样也传入source_mask，但是进行源数据遮掩的原因并非是抑制信息泄露，而是遮蔽掉对结果没有意义的padding。
        """
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        """
        最后一个子层就是前馈全连接子层，经过它的处理后就可以返回结果，这就是我们的解码器结构
        """
        z = self.sublayer[2](x, self.feed_forward)
        return z


# 3.1.6 Mask
def subsequent_mask(size):
    """
    生成向后遮掩的掩码张量->形成一个三角矩阵
    :param size: 掩码张量最后两个维度的大小, 最后两维形成一个方阵
    """
    attn_shape = (1, size, size)

    # 然后使用np.ones()向这个形状中添加1元素，np.triu()形成上三角阵
    mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')

    # 最后将numpy类型转化为torch中的tensor，内部做一个1- 的操作。这个其实是做了一个三角阵的反转，subsequent_mask中的每个元素都会被1减。
    # 如果是0，subsequent_mask中的该位置由0变成1
    # 如果是1，subsequent_mask中的该位置由1变成0
    return torch.from_numpy(mask) == 0


# 3.2 Attention

# 3.2.1 Scaled Dot-Product Attention
def attention(query, key, value, mask=None, dropout=None):
    """
    实现了缩放点积注意力
    1. 首先取query的最后一维的大小，对应词嵌入维度
    2. 利用公式计算注意力分数scores, 这里面key是将最后两个维度进行转置 -> (句子长度维度,词(多头)向量维度)
    3. 判断是否使用掩码张量
    4. 对scores的最后一维进行softmax操作，获得最终的注意力张量
    5. 判断是否使用dropout进行随机置0
    6. 最后，将p_attn与value张量相乘获得最终的query注意力表示，同时返回注意力张量
    """
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        # 将掩码张量和scores张量每个位置一一比较
        # 如果掩码张量则对应的scores张量相同，则用-1e9来替换
        scores = scores.masked_fill(mask == 0, value=-1e9)

    p_attn = F.softmax(scores, dim=-1)

    if dropout is not None:
        p_attn = dropout(p_attn)

    attn = torch.matmul(p_attn, value)
    return attn, p_attn


# 3.2.2 Multi-Head Attention


class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        """
        :param n_heads: 注意力头数
        :param d_model: 词嵌入维度
        :param dropout: 比率默认为0.1
        """
        super(MultiHeadedAttention, self).__init__()
        # 判断n_heads是否能被d_model整除 -> embedding_dim / n_heads
        assert d_model % n_heads == 0
        self.d_k = d_model // n_heads  # 512//8=64
        self.h = n_heads  # 8

        # 创建linear层，并且克隆4个 -> Q,K,V各一个，最后拼接的矩阵还需要一个
        self.linear = clone(module=nn.Linear(d_model, d_model), N=4)  # 512*512
        self.p_attn = None  # 代表最后得到的注意力张量
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        """
        1. 从 d_model(512) --> h*d_k(8*64) 批量执行所有线性投影
        2. 将注意力集中在所有投影向量上
        3. Concat并最终应用到线性层
        """
        if mask is not None:
            mask = mask.unsqueeze(1)  # 拓展维度，表示多头中的第n头
        n_batches = query.size(0)  # batch_size代表有多少条样本

        """
        1. 首先利用zip将输入QKV与三个线性层组到一起，然后利用for循环，将输入QKV分别传到线性层中,
           使用view()对线性变换的结构进行维度重塑，为每个头分割输入
               多加了一个维度h代表头，这样就意味着每个头可以获得一部分词特征组成的句子
               其中的-1代表自适应维度，即m句子长度维度，将自动计算这里的值
           然后对第二维和第三维进行转置操作：
               原因：为了让代表句子长度维度和词向量维度能够相邻，这样注意力机制才能找到词义与句子位置的关系，
               从attention函数中可以看到，利用的是原始输入的倒数第一和第二维，这样我们就得到了每个头的输入
        """
        query, key, value = [
            lin(x).view(n_batches, -1, self.h, self.d_k).transpose(1, 2)  # -1 <-> self.h
            for lin, x in zip(self.linear, (query, key, value))
        ]
        """
        2. 得到每个头的输入后，接下来就是将他们传入到attention中，
           这里直接调用我们之前实现的attention函数，同时也将mask和dropout传入其中
        """
        attn, self.p_attn = attention(query, key, value, mask, self.dropout)
        """
        3. 通过多头注意力计算后，我们就得到了每个头计算结果组成的4维张量，我们需要将其转换为输入的形状以方便后续的计算，
           因此这里开始进行第一步处理环节的逆操作，先对第二和第三维进行转置，
           然后使用contiguous(): 能够让转置后的张量应用view()，否则将无法直接使用.
           下一步就是使用view重塑形状，变成和输入形状相同。  
           最后使用线性层列表中的最后一个线性变换得到最终的多头注意力结构的输出
        """
        concat = attn.transpose(1, 2).contiguous().view(n_batches, -1, self.h * self.d_k)
        x = self.linear[-1](concat)

        return x


# 3.3 Position-wise Feed-Forward Networks
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        :param d_model: 通过前馈全连接层后输入和输出的维度不变
        :param d_ff: 内部维度：第二个线性层的输入维度和第一个线性层的输出
        """
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        首先经过第一个线性层，然后使用F中的relu函数进行激活，
        之后再使用dropout进行随机置0，最后通过第二个线性层w2，返回最终结果
        """
        x = self.w_2(self.dropout(F.relu(self.w_1(x))))
        return x


# 3.4 Embeddings and Softmax
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        """
        :param x: 这里代表输入给模型的单词文本通过词表映射后的one-hot向量
        :return: 将x传给self.lut并与根号下self.d_model相乘作为结果返回
        """
        return self.lut(x) * math.sqrt(self.d_model)


# 3.5 Positional Encoding
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout, max_len=5000):
        """
        :param d_model: 词嵌入维度 这里是512维
        :param dropout: 词嵌入维度
        :param max_len: 每个句子的最大长度
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        # 使用与原公式等价的表示
        # 目的是避免中间的数值计算结果超出float的范围
        pos_embed = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)  # 0->4999 再插入一个维度(5000,1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )  # shape=[256]
        # div_term 实现的是分母
        # pe[:, 0::2] 表示第二个维度从 0 开始以间隔为 2 取值，即偶数。
        pos_embed[:, ::2] = torch.sin(position * div_term)  # shape=[max_len, 256]
        pos_embed[:, 1::2] = torch.cos(position * div_term)

        pos_embed = pos_embed.unsqueeze(0)  # shape=[1, 500, 512]
        self.register_buffer('pe', pos_embed)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)


# 3.6 Full Model
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    """
    构建模型
    :param src_vocab: 输入词表大小
    :param tgt_vocab: 目标词表大小
    :param N: 编码器和解码器堆叠的基础模块个数
    :param d_model: 词嵌入的维度
    :param d_ff: 逐位置的前馈网络中的内部维度
    :param h: 注意力头的个数
    :param dropout:
    """
    c = copy.deepcopy
    attn = MultiHeadedAttention(n_heads=h, d_model=d_model, dropout=dropout)
    ff = PositionwiseFeedForward(d_model=d_model, d_ff=d_ff, dropout=dropout)
    position = PositionalEncoding(d_model=d_model, dropout=dropout)
    # -------
    encoderLayer = EncoderLayer(size=d_model, self_attention=c(attn), feed_forward=c(ff), dropout=dropout)
    decoderLayer = DecoderLayer(size=d_model, self_attention=c(attn), src_attn=c(attn), feed_forward=c(ff),
                                dropout=dropout)
    srcEmbed = Embeddings(d_model=d_model, vocab=src_vocab)
    tgtEmbed = Embeddings(d_model=d_model, vocab=tgt_vocab)
    generator = Generator(d_model=d_model, vocab=tgt_vocab)
    # -------

    model = EncoderDecoder(
        encoder=Encoder(layer=encoderLayer, N=N),
        decoder=Decoder(layer=decoderLayer, N=N),
        src_embed=nn.Sequential(srcEmbed, c(position)),
        tgt_embed=nn.Sequential(tgtEmbed, c(position)),
        generator=generator
    )
    # 初始化参数: 使用Glorot初始化: 1/𝑓𝑎𝑛_𝑎𝑣𝑔, 𝑓𝑎𝑛_𝑎𝑣𝑔=(𝑓𝑎𝑛_𝑖𝑛 +𝑓𝑎𝑛_𝑜𝑢𝑡)/2
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model



In [3]:
import time
import pandas as pd

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR


import altair as alt


# 5 Training
# 5.1 Batched and Masking
class Batch:
    def __init__(self, src, tgt, pad=2):
        """
        :param pad: 默认2 表示<blank>
        """
        self.src = src
        # 将与令牌匹配的位置表示为False, 否则为True
        # 并在倒数第二个维度后面添加一维度
        self.src_mask = (src != pad).unsqueeze(-2)

        if tgt is not None:
            self.tgt = tgt[:, :-1]  # Decoder的输入，即除去最后一个结束token的部分
            self.tgt_y = tgt[:, 1:]  # Decoder的期望输入，即除去首个一个起始token的部分
            self.tgt_mask = self.make_std_mask(self.tgt, pad)
            self.ntokens = (self.tgt_y != pad).data.sum()  # 所有True的词元数量

    @staticmethod
    # staticmethod 返回函数的静态方法 可以不实例化即可调用方法
    def make_std_mask(tgt, pad):
        """
        pad 和 future words 均在mask中用pad表示
        """
        tgt_mask = (tgt != pad).unsqueeze(-2)
        sequence_len = tgt.size(-1)  # 或是batch中最长时间步数
        tgt_mask = tgt_mask & subsequent_mask(size=sequence_len).type_as(
            tgt_mask.data
            # &:进行位运算
            # subsequent_mask()返回维度为(1, size, size)
            # type_as():将数据类型转换为tgt_mask的数据类型
        )
        return tgt_mask


# 5.2 Training Loop
class TrainState:
    """
    跟踪处理的步骤、示例和标记的数量
    """
    step: int = 0  # 当前epoch的步
    accum_step: int = 0  # 梯度累积步数
    samples: int = 0  # 使用的示例总数
    tokens: int = 0  # 处理的tokens总数


def run_epoch(data_iter, model, loss_compute,
              optimizer, scheduler,
              mode="train", accum_iter=1,
              train_state=TrainState(),
              device=None):
    """
    完成了一个epoch训练的所有工作
    包括数据加载、模型推理、损失计算与方向传播，同时将训练过程信息进行打印
    """
    # 训练单个epoch
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0  # 梯度累积步数
    for i, batch in enumerate(data_iter):
        # model是一个EncoderDecoder对象
        # 前向传播将src, src_mask传入编码函数，得到结果后与src_mask, tgt和tgt_mask一同传给解码函数
        out = model.forward(src=batch.src, tgt=batch.tgt, src_mask=batch.src_mask, tgt_mask=batch.tgt_mask)
        # 梯度累加技术 loss_node = loss_node / accum_iter
        # accum_iter:小批次数 默认是1 不使用梯度累加技术
        loss, loss_node = loss_compute(x=out, y=batch.tgt_y, norm=batch.ntokens)  # 计算损失->SimpleLossCompute
        if mode == "train" or mode == "train+log":
            loss_node.backward()  # 反向传播->不进行梯度清零, 执行梯度累加的操作
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:  # 梯度累加达到固定次数之后
                optimizer.step()  # 更新参数
                optimizer.zero_grad(set_to_none=True)  # 梯度清零
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()

        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens

        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]  # 获取学习率
            elapsed = time.time() - start  # 计算40个迭代所需时间
            print(("Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f " +
                   "| Tokens / Sec: %7.1f | Learning Rate: %6.1e") %
                  (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr))
            start = time.time()
            tokens = 0

        del loss
        del loss_node
    return total_loss / total_tokens, train_state


# 5.2 Optimizer
def rate(step, model_size, factor, warmup):
    """
    对于 Lambda LR 函数，我们必须将步骤默认为 1 避免零提升为负幂。
    :param step: 时间步长
    :param model_size: 模型维度
    :param factor: 示例中为1
    :param warmup: 预热迭代数
    :return:
    """
    if step == 0:
        step = 1
    return factor * (
            model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )


def example_learning_schedule():
    """
    学习率调度示例: 在 opts 列表中有 3 个示例。
    为每个示例运行 20000 个 epoch
    学习率调度使用 自定义调整学习率LambdaLR
    数据可视化工具: Altair
    """
    opts = [
        [512, 1, 4000],  # example 1
        [512, 1, 8000],  # example 2
        [256, 1, 4000],  # example 3
    ]
    dummy_model = torch.nn.Linear(1, 1)
    learning_rates = []

    for idx, example in enumerate(opts):
        optimizer = torch.optim.Adam(dummy_model.parameters(),
                                     lr=1,
                                     betas=(0.9, 0.98),
                                     eps=1e-9)
        lr_scheduler = LambdaLR(
            optimizer=optimizer,
            lr_lambda=lambda step: rate(step, *example))
        tmp = []
        #  采取20000次的虚拟训练步骤，并保存每一步的学习率
        for step in range(20000):
            # optimizer.param_groups[0]：长度为6的字典，
            # 包括[‘amsgrad’, ‘params’, ‘lr’, ‘betas’, ‘weight_decay’, ‘eps’]
            tmp.append(optimizer.param_groups[0]["lr"])
            optimizer.step()  # 更新参数
            lr_scheduler.step()  # 更新参数
        learning_rates.append(tmp)

    learning_rates = torch.tensor(learning_rates)
    # ----数据可视化----
    # 使 altair 能够处理超过 5000 行
    alt.data_transformers.disable_max_rows()

    opts_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Learning Rate": learning_rates[warmup_idx, :],
                    "model_size:warmup": ["512:4000", "512:8000", "256:4000"][
                        warmup_idx
                    ],
                    "step": range(20000),
                }
            )
            for warmup_idx in [0, 1, 2]
        ]
    )
    return (
        alt.Chart(opts_data)
            .mark_line()
            .properties(width=600)
            .encode(x="step", y="Learning Rate", color="model_size:warmup:N")
            .interactive()
    )


# 5.3 Regularization

# 5.3.2 Label Smoothing
class LabelSmoothing(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum")
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())


RUN_EXAMPLES = True


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def example_label_smoothing():
    crit = LabelSmoothing(5, 0, 0.4)
    predict = torch.FloatTensor(
        [
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
            [0, 0.2, 0.7, 0.1, 0],
        ]
    )
    crit(x=predict.log(), target=torch.LongTensor([2, 1, 0, 3, 3]))
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "target distribution": crit.true_dist[x, y].flatten(),
                    "columns": y,
                    "rows": x,
                }
            )
            for y in range(5)
            for x in range(5)
        ]
    )

    return (
        alt.Chart(LS_data)
            .mark_rect(color="Blue", opacity=1)
            .properties(height=200, width=200)
            .encode(
            alt.X("columns:O", title=None),
            alt.Y("rows:O", title=None),
            alt.Color(
                "target distribution:Q", scale=alt.Scale(scheme="viridis")
            ),
        )
            .interactive()
    )


def loss(x, crit):
    d = x + 3 * 1
    predict = torch.FloatTensor([[0, x / d, 1 / d, 1 / d, 1 / d]])
    return crit(predict.log(), torch.LongTensor([1])).data


def penalization_visualization():
    crit = LabelSmoothing(5, 0, 0.1)
    loss_data = pd.DataFrame({
        "Loss": [loss(x, crit) for x in range(1, 100)],
        "Steps": list(range(99)),
    }).astype("float")

    return (alt.Chart(loss_data).mark_line().properties(width=350).encode(
        x="Steps",
        y="Loss",
    ).interactive())


# 6 A First Example

# 6.1 Synthetic Data
def data_gen(V, n_batches, batch_size, s_len=10, device=None):
    """
    <编码器-解码器数据复制任务> 随机数据生成器
    :param device: 是否使用GPU加速
    :param V: 词典数量，取值范围[0, V-1]，约定0作为特殊符号使用代表padding
    :param batch_size: 批次大小
    :param n_batches: 需要生成的批次数量
    :param s_len: 生成的序列数据的长度
    """
    for i in range(n_batches):
        src_data = torch.randint(2, V, size=(batch_size, s_len))
        # 约定输出为输入除去序列第一个元素，即向后平移一位进行输出，同时输出数据要在第一个时间步添加一个起始符
        tgt_data = src_data.clone()
        tgt_data[:, 0] = 1  # 将序列的第一个时间步置为1(即约定的起始符)
        # .batch()
        # 返回一个新的tensor，从当前计算图中分离下来的，但是仍指向原变量的存放位置
        # 不同之处只是requires_grad为false，得到的这个tensor永远不需要计算其梯度，不具有grad。
        # requires_grad 默认为False
        src = src_data.requires_grad_(False).clone().detach()
        tgt = tgt_data.requires_grad_(False).clone().detach()
        if device == "cuda":
            src = src.cuda()
            tgt = tgt.cuda()
        yield Batch(src=src, tgt=tgt, pad=0)


# 6.2 Loss Computation
class SimpleLossCompute:
    def __init__(self, generator, criterion):
        self.generator = generator
        self.criterion = criterion  # 使用标签平滑

    def __call__(self, x, y, norm):
        """
        :param x: decoder输出的结果
        :param y: 标签数据
        :param norm: loss的归一化系数，用batch中所有有效token数即可
        """
        x = self.generator(x)
        # contiguous():
        # 1. 由于torch.view等方法操作需要连续的Tensor
        # 2. 出于性能考虑 使用该方法后会重新据开辟一块内存空间保证数是在逻辑顺序和内存中是一致的
        x_ = x.contiguous().view(-1, x.size(-1))
        y_ = y.contiguous().view(-1)
        loss = self.criterion(x_, y_)
        sloss = (loss / norm)

        return sloss.data * norm, loss


# 6.3 Greedy Decoding
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    # encoder()编码函数: 使用src_embed对src做处理，然后和src_mask一起传给self.encoder
    memory = model.encode(src=src, src_mask=src_mask)
    # ys代表目前已生成的序列，最初为仅包含一个起始符的序列，不断将预测结果追加到序列最后
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        # decoder()解码函数: 使用tgt_embed对tgt做处理，然后和src_mask, tgt_mask, memory一起传给self.decoder
        out = model.decode(memory=memory,
                           src_mask=src_mask,
                           tgt=ys,
                           tgt_mask=subsequent_mask(size=ys.size(1)).type_as(src.data))
        # generator: 类别生成器对象 -> linear+softmax
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        # cat(): 实现拼接操作
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)],
            dim=1)
    return ys


# 6.4 Training Example
# def execute_example(fn, args=[]):
#     if __name__ == "__main__" and RUN_EXAMPLES:
#         fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None


def example_simple_model(device=None):
    V = 11  # 字典的大小
    criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
    model = make_model(src_vocab=V, tgt_vocab=V, N=2)
    if device == "cuda":
        model.cuda()
    model_size = model.src_embed[0].d_model  # 512

    n_epochs = 20
    n_batch_train_epoch = 20  # 训练时每个epoch所需批次大小
    n_batch_val_epoch = 5  # 验证时每个epoch所需批次大小
    batch_size = 80

    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=0.5,
                                 betas=(0.9, 0.98),
                                 eps=1e-9)
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(step=step, model_size=model_size, factor=0.1, warmup=400)
    )

    for epoch in range(n_epochs):
        loss_compute = SimpleLossCompute(generator=model.generator,
                                         criterion=criterion)

        print(f"\n|   批次: {epoch}   |")
        print("*" * 5 + "训练" + "*" * 5)
        model.train()  # self.training=True

        train_data_iter = data_gen(V=V, n_batches=n_batch_train_epoch,
                                   batch_size=batch_size, device=device)
        run_epoch(data_iter=train_data_iter,
                  model=model,
                  loss_compute=loss_compute,
                  optimizer=optimizer,
                  scheduler=lr_scheduler,
                  mode="train")

        # -----------
        print("*" * 5 + "验证" + "*" * 5)
        model.eval()  # self.training=False

        val_data_iter = data_gen(V=V, n_batches=n_batch_val_epoch,
                                 batch_size=batch_size, device=device)
        valid_mean_loss = run_epoch(data_iter=val_data_iter,
                                    model=model,
                                    loss_compute=loss_compute,
                                    optimizer=DummyOptimizer(),  # None
                                    scheduler=DummyScheduler(),  # None
                                    mode="eval")[0]  # 返回: total_loss / total_tokens
        print(f"|验证损失: {valid_mean_loss} |")

    model.eval()
    torch.save(model, './models/Pytorch/example_1_copy.pth')



In [4]:
pip install GPUtil

Note: you may need to restart the kernel to use updated packages.


In [5]:

# ----
from torchtext import data, datasets
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import GPUtil
import spacy
import os
from os.path import exists




# 1. 加载德文，英文分词器
def load_tokenizers():
    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en


# 2. 分词
def tokenize(text, tokenizer):
    """
    分词处理
    Spacy 会先将文档分解成句子，然后再 tokenize 。我们可以使用迭代来遍历整个文档
    :param text: 文本
    :param tokenizer: 分词器 spacy_zh 或 spacy_en
    :return:
    """
    return [token.text for token in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    # index[0]:德文 index[1]:英文
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])


# 3. 构建词汇表
def build_vocabulary(spacy_de, spacy_en):
    """
    构建数据集
    - 使用torchtext自带的机器翻译数据集 Multi30k
        -- language_pair:指定使用的翻译句子对的语言，默认是从德语到英语。数据集中的每一行是一对指定语言的句子对
    - build_vocab_from_iterator() :从迭代器构建词汇函数
        -- iterator: 构建 Vocab 的迭代器
        -- min_freq: 在词汇表中包含标记所需的最小频率
        -- specials: 要添加的特殊符号
    """
    BOS_WORD = '<s>'  # beginning of sequence 序列开始标识
    EOS_WORD = '</s>'  # end of sequence 序列结束标识
    BLANK_WORD = '<blank>'  # 空白标识
    UNK_WORD = '<unk>'  # 未知字符标识

    def tokenize_de(text):
        return tokenize(text=text, tokenizer=spacy_de)

    def tokenize_en(text):
        return tokenize(text=text, tokenizer=spacy_en)

    print("***构建德文数据集***")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        iterator=yield_tokens(
            data_iter=train + val + test,
            tokenizer=tokenize_de,
            index=0
        ),
        min_freq=2,
        specials=[BOS_WORD, EOS_WORD, BLANK_WORD, UNK_WORD]
    )

    print("***构建英文数据集***")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        iterator=yield_tokens(
            data_iter=train + val + test,
            tokenizer=tokenize_en,
            index=1
        ),
        min_freq=2,
        specials=[BOS_WORD, EOS_WORD, BLANK_WORD, UNK_WORD]
    )

    vocab_src.set_default_index(vocab_src[UNK_WORD])
    vocab_tgt.set_default_index(vocab_src[UNK_WORD])

    return vocab_src, vocab_tgt


vocab_path = "./vocab.pt"


def load_vocab(spacy_de, spacy_en, vocab_path):
    if not exists(vocab_path):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), vocab_path)
        print("完成构建!")
    else:
        vocab_src, vocab_tgt = torch.load(vocab_path)
        print("完成加载!")
    print("德文词汇量：" + str(len(vocab_src)))
    print("英文词汇量：" + str(len(vocab_tgt)))
    return vocab_src, vocab_tgt


# 4. 迭代器
def collate_batch(batch, src_pipeline, tgt_pipeline, src_vocab, tgt_vocab, device, max_padding=128, PAD_id=2):
    """
    批次数据整理:标识开始结束token 并进行填充至统一长度
    :param batch:
    :param src_pipeline: 输入分词器-tokenize_de
    :param tgt_pipeline: 目标分词器-tokenize_en
    :param src_vocab: 输入词汇表-vocab_src
    :param tgt_vocab: 输出词汇表-vocab_tgt
    :param device: 使用GPU加速
    :param max_padding: 最大填充默认128
    :param PAD_id: 填充id -> <black> 空白标识
    :return:
    """
    BOS_id = torch.tensor([0], device=device)  # <s>  序列开始标识ID
    EOS_id = torch.tensor([1], device=device)  # </s> 序列结束标识ID
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        # 对输入批次进行预处理 添加序列开始和结束标识
        processed_src = torch.cat(
            [
                BOS_id,  # <s>
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                EOS_id,  # </s>
            ],
            dim=0,
        )
        # 对目标批次进行预处理
        processed_tgt = torch.cat(
            [
                BOS_id,  # <s>
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                EOS_id,  # </s>
            ],
            dim=0,
        )

        # F.pad 通过填充较短的序列来处理，以便批次中的所有序列具有相同的长度
        src_list.append(
            F.pad(
                input=processed_src,
                pad=(0, max_padding - len(processed_src)),  # (0, 128-len(processed_src))
                mode="constant",
                value=PAD_id,
            ))
        tgt_list.append(
            F.pad(
                input=processed_tgt,
                pad=(0, max_padding - len(processed_tgt)),
                mode="constant",
                value=PAD_id,
            ))

    # stack(): 对张量序列进行连接
    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)

    return (src, tgt)


def create_dataloaders(device, vocab_src, vocab_tgt, spacy_de, spacy_en, batch_size=12000, max_padding=128,
                       is_distributed=True):
    """
    创建数据加载器
    :param spacy_de: 德文分词器
    :param spacy_en: 英文分词器
    :param batch_size: 批次大小为12000
    :param is_distributed: 是否使用分布式训练
    :return:
    """

    def tokenize_de(text):
        return tokenize(text=text, tokenizer=spacy_de)

    def tokenize_en(text):
        return tokenize(text=text, tokenizer=spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch=batch,
            src_pipeline=tokenize_de,
            tgt_pipeline=tokenize_en,
            src_vocab=vocab_src,
            tgt_vocab=vocab_tgt,
            device=device,
            max_padding=max_padding,
            # get_stoi(): 字典将标记映射到索引
            PAD_id=vocab_src.get_stoi()["<blank>"],  # 2
        )

    train_iter, valid_iter, test_iter = datasets.Multi30k(language_pair=("de", "en"))

    # 1. 转换数据集类型为map-style
    #   - to_map_style_dataset(): 将`iterable-style`数据集转换为`map-style`数据集。
    #       - `map-style`是使用索引/键向数据样本进行映射
    #       - `iterable-style`的迭代型的数据集就是真正载入数据
    train_iter_map = to_map_style_dataset(train_iter)
    valid_iter_map = to_map_style_dataset(valid_iter)

    # 2. 使用分布式采样器
    #   - DistributedSampler(): 分布式采样器  由于使用多GPU训练 加载策略是负责只提供加载数据集中的一个子集
    #       - 使用分布式采样器需要数据集的len()
    train_sampler = (
        DistributedSampler(dataset=train_iter_map) if is_distributed else None
    )
    valid_sampler = (
        DistributedSampler(dataset=valid_iter_map) if is_distributed else None
    )

    # 3. 创建数据加载器： 使用DataLoader()结合数据集和采样器，并提供可迭代的给定的数据集。
    train_dataloader = DataLoader(
        dataset=train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),  # 如果未指定采样器则进行混洗
        sampler=train_sampler,
        collate_fn=collate_fn,  # 在使用批量加载`map-style`数据集时使用 批次数据整理
    )
    valid_dataloader = DataLoader(
        dataset=valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader


# 5. 多 GPU 训练

def train_worker(gpu, ngpus_per_node, vocab_src, vocab_tgt, spacy_de, spacy_en, config, is_distributed=False):
    """
    配置训练任务
    :param gpu: 主机编号
    :param ngpus_per_node: 主机数量
    :param config: 参数配置
    :param is_distributed: 是否使用分布式训练
    :return:
    """
    # ----多卡配置----
    print(f"使用 GPU 训练工作进程： {gpu} 训练", flush=True)
    torch.cuda.set_device(gpu)

    pad_idx = vocab_tgt["<blank>"]  # 空白填充token的id
    d_model = 512
    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.cuda(gpu)
    module = model
    is_main_process = True

    if is_distributed:
        """
        torch.distributed库: 实现单机多卡训练库
        - init_process_group(): 初始化默认的分布式进程组
           - backend: 一般来说使用NCCL对于GPU分布式训练，使用gloo对CPU进行分布式训练
           - init_method: URL指定了如何初始化互相通信的进程    
           - rank: 优先度或gpu的编号/进程的编号 rank=0的主机就是主要节点
           - world_size: 执行训练的所有的进程数/GPU数
        - DistributedDataParallel(): DDP模式: 使用多进程；性能更优；模型广播只在初始化的时候, 故训练加速
        """
        dist.init_process_group(
            backend="nccl",
            init_method="env://",
            rank=gpu,
            world_size=ngpus_per_node
        )
        model = DDP(module=model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    # ----训练配置----
    criterion = LabelSmoothing(
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu)
    # 创建数据加载器
    train_dataloader, valid_dataloader = create_dataloaders(
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,  # 批次大小 // 总GPU数
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
    )
    # 优化器
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=config["base_lr"],
                                 betas=(0.9, 0.98),
                                 eps=1e-9)
    # 学习率调度
    lr_scheduler = LambdaLR(
        optimizer=optimizer,
        lr_lambda=lambda step: rate(step, d_model, factor=1, warmup=config["warmup"]),
    )
    train_state = TrainState()  # 跟踪处理的步骤、示例和标记的数量

    for epoch in range(config["num_epochs"]):
        if is_distributed:  # 使用分布式训练就需要进行分批操作
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        print(f"\n|   批次: {epoch}   |")
        print("*" * 5 + "训练" + "*" * 5)
        model.train()
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        train_data_iter = (Batch(src=b[0], tgt=b[1], pad=pad_idx) for b in train_dataloader)  #######
        _, train_state = run_epoch(
            data_iter=train_data_iter,
            model=model,
            loss_compute=SimpleLossCompute(module.generator, criterion),
            optimizer=optimizer,
            scheduler=lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        GPUtil.showUtilization()  # 实时查看GPU状况
        # 保存检查点模型
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        # -----------
        print("*" * 5 + "验证" + "*" * 5)
        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        valid_data_iter = (Batch(src=b[0], tgt=b[1], pad=pad_idx) for b in valid_dataloader)  ######
        valid_mean_loss = run_epoch(
            data_iter=valid_data_iter,
            model=model,
            loss_compute=SimpleLossCompute(module.generator, criterion),
            optimizer=DummyOptimizer(),
            scheduler=DummyScheduler(),
            mode="eval",
        )
        print(valid_mean_loss)
        torch.cuda.empty_cache()

    # 保存最终模型
    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path)


def train_distributed_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    """
    配置分布式训练任务
    """
    ngpus = torch.cuda.device_count()
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12356"
    print(f"检测到的 GPUs 数量： {ngpus}")
    print("产生训练过程中 ...")

    # torch.multiprocessing(): 实现pytorch多进程
    mp.spawn(
        fn=train_worker,
        nprocs=ngpus,
        args=(ngpus, vocab_src, vocab_tgt, spacy_de, spacy_en, config, True),
    )


def train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    """
    选择训练任务的训练类型
    """
    if config["distributed"]:  # 执行分布式训练
        train_distributed_model(
            vocab_src, vocab_tgt, spacy_de, spacy_en, config
        )
    else:   # 执行单GPU训练
        train_worker(
            gpu=0,
            ngpus_per_node=1,
            vocab_src=vocab_src,
            vocab_tgt=vocab_tgt,
            spacy_de=spacy_de,
            spacy_en=spacy_en,
            config=config,
            is_distributed=False
        )


def load_trained_model():
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 8,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt"
    if not exists(model_path):
        train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config)

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(torch.load("multi30k_model_final.pt"))
    return model


aaa

In [6]:
if __name__ == "__main__":
    spacy_de, spacy_en = load_tokenizers()
    vocab_path = "./vocab.pt"
    vocab_src, vocab_tgt = load_vocab(spacy_de, spacy_en, vocab_path)
    model = load_trained_model()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 20.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


***构建德文数据集***
***构建英文数据集***


/opt/conda/lib/python3.7/site-packages/torch/utils/data/datapipes/iter/combining.py:181: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  "the buffer and each child DataPipe will read from the start again.", UserWarning)


完成构建!
德文词汇量：8315
英文词汇量：6384
使用 GPU 训练工作进程： 0 训练

|   批次: 0   |
*****训练*****
[GPU0] Epoch 0 Training ====
Epoch Step:      1 | Accumulation Step:   1 | Loss:   7.58 | Tokens / Sec:   825.4 | Learning Rate: 5.4e-07
Epoch Step:     41 | Accumulation Step:   5 | Loss:   7.36 | Tokens / Sec:  2880.7 | Learning Rate: 1.1e-05
Epoch Step:     81 | Accumulation Step:   9 | Loss:   6.98 | Tokens / Sec:  2856.0 | Learning Rate: 2.2e-05
Epoch Step:    121 | Accumulation Step:  13 | Loss:   6.71 | Tokens / Sec:  2825.3 | Learning Rate: 3.3e-05
Epoch Step:    161 | Accumulation Step:  17 | Loss:   6.45 | Tokens / Sec:  2841.8 | Learning Rate: 4.4e-05
Epoch Step:    201 | Accumulation Step:  21 | Loss:   6.38 | Tokens / Sec:  2801.3 | Learning Rate: 5.4e-05
Epoch Step:    241 | Accumulation Step:  25 | Loss:   6.23 | Tokens / Sec:  2863.9 | Learning Rate: 6.5e-05
Epoch Step:    281 | Accumulation Step:  29 | Loss:   5.95 | Tokens / Sec:  2808.8 | Learning Rate: 7.6e-05
Epoch Step:    321 | Accumulati